> # Fine-tune mô hình PhoBERT để phân loại cảm xúc đánh giá MoMo

Trong notebook này, chúng ta sẽ thực hiện fine-tune mô hình PhoBERT (phiên bản base) cho bài toán phân loại cảm xúc (tích cực/trung lập/tiêu cực) trên tập dữ liệu đánh giá phim từ MoMo. Tập dữ liệu đầu vào là file CSV momo_reviews_balanced.csv với số lượng nhãn cân bằng giữa ba loại cảm xúc. Các bước chính bao gồm:
- Tiền xử lý dữ liệu: Tạo thêm phiên bản không dấu của nội dung đánh giá bên cạnh phiên bản có dấu gốc để tăng dữ liệu huấn luyện.

- Chia tập dữ liệu: Tách dữ liệu thành tập huấn luyện, tập validation và tập kiểm tra theo tỷ lệ 80/10/10 (giữ cân bằng giữa các nhãn).

- Thiết lập mô hình: Sử dụng mô hình PhoBERT base từ thư viện Huggingface Transformers cho bài toán phân loại. Chuẩn bị tokenizer tương ứng.

- Huấn luyện mô hình: Sử dụng GPU nếu có để tăng tốc. Áp dụng các biện pháp tránh overfitting như Dropout, Weight Decay (Regularization) và Early Stopping khi huấn luyện. Kết hợp cả dữ liệu có dấu và không dấu trong quá trình huấn luyện.

- Theo dõi quá trình: Báo cáo độ chính xác (accuracy) và hàm mất mát (loss) trên tập validation sau mỗi epoch để đánh giá quá trình huấn luyện.
Lưu và đánh giá mô hình: Lưu lại mô hình tốt nhất. Đánh giá mô hình trên tập kiểm tra và xuất kết quả độ chính xác cuối cùng.

Lưu ý: PhoBERT được huấn luyện trên văn bản đã được tách từ (word segmentation), tức là các từ nhiều âm tiết cần được nối bằng dấu gạch dưới trước khi token hóa ​
[github.com](https://github.com/huggingface/transformers/blob/main/docs/source/en/model_doc/phobert.md)
. Trong phạm vi notebook này, trong thực tế có sử dụng thư viện như Underthesea để tách từ cho văn bản tiếng Việt nhằm đạt kết quả tốt nhất.

## Cài đặt và import các thư viện cần thiết

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


  Đầu tiên, cài đặt các thư viện cần dùng: transformers và datasets từ Huggingface, underthesea cho việc tách từ (nếu cần), và các thư viện phổ biến như Pandas, scikit-learn. Sau đó tiến hành import các module cần thiết. (Nếu môi trường đã có sẵn các thư viện này thì có thể bỏ qua bước cài đặt.)

In [ ]:
!pip install transformers datasets underthesea scikit-learn

In [ ]:
!pip install --upgrade torchvision

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
# # Thư viện underthesea (nếu muốn tách từ) cũng có thể import ở đây
# # from underthesea import word_tokenize


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

# Tắt Weights & Biases (wandb) logging
import os
os.environ["WANDB_DISABLED"] = "true"


## Bước 1: Chuẩn bị dữ liệu

### 1.1 Đọc dữ liệu và khám phá tổng quan

  Đọc file CSV chứa dữ liệu đánh giá. Mỗi mẫu dữ liệu gồm nội dung đánh giá và nhãn cảm xúc tương ứng (Tiêu cực, Trung lập, Tích cực). Ta sử dụng Pandas để tải dữ liệu và xem qua vài dòng đầu cũng như phân bố các nhãn.

In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv('momo_reviews_balanced.csv')
# Đổi tên cột nhãn cho rõ ràng (vd: 'label' -> 'label_text')
df.rename(columns={'label': 'label_text'}, inplace=True)
print("Số lượng mẫu:", len(df))
print("Các nhãn và số lượng mẫu mỗi nhãn:")
print(df['label_text'].value_counts())
# Xem 5 dòng dữ liệu đầu tiên
df.head(5)

Số lượng mẫu: 14403
Các nhãn và số lượng mẫu mỗi nhãn:
label_text
Trung lập    4801
Tiêu cực     4801
Tích cực     4801
Name: count, dtype: int64


,Tên phim,Người đánh giá,Điểm,Nhãn cảm xúc,Nội dung đánh giá,Noi_dung_sach,Noi_dung_sach_giu_dau,label_text
0,Review phim Panor: Tà Thuật Huyết Ngải trên MoMo,Ninh Thị Tuyết,5/10,Tạm ổn,Coi rất cuon nha,coi rat cuon nha,coi rất cuon nha,Trung lập
1,Review phim Cô Dâu Hào Môn trên MoMo,Nguyễn Ngọc Phúc,1/10,Kén người mê,"Vô bổ, phí tiền, luẩn cuẩn, k ra cái thể loại ...",vo bo phi tien luan cuan không ra cai the loai...,vô bổ phí tiền luẩn cuẩn không ra cái thể loại...,Tiêu cực
2,Review phim Cô Dâu Hào Môn trên MoMo,Phạm Thị Yến Nhi,2/10,Kén người mê,"Phim kịch bản lòng vòng, đầu đuôi chuột, bất h...",phim kich ban long vong dau duoi chuot bat hop...,phim kịch bản lòng vòng đầu đuôi chuột bất hợp...,Tiêu cực
3,Review phim Mật Vụ Phụ Hồ trên MoMo,Nguyễn Trọng Huy,6/10,Tạm ổn,"Phim bình thường, buff quá tay",phim binh thuong buff qua tay,phim bình thường buff quá tay,Trung lập
4,Review phim Gặp Lại Chị Bầu trên MoMo,Võ Kim Ngân,2/10,Kén người mê,"Theo ý kiến riêng của mình, điểm cộng phim là ...",theo kien rieng cua minh diem cong phim la dan...,theo kiến riêng của mình điểm cộng phim là dàn...,Tiêu cực


### 1.2 Tiền xử lý dữ liệu văn bản

  PhoBERT yêu cầu dữ liệu tiếng Việt có dấu. Tuy nhiên trong thực tế người dùng có thể nhập văn bản không dấu, do đó ta bổ sung phiên bản không dấu của mỗi đánh giá để mô hình được huấn luyện trên cả hai trường hợp. Cụ thể, với mỗi mẫu đánh giá gốc (có dấu), ta tạo thêm một bản sao nội dung đã được loại bỏ dấu tiếng Việt. Để loại bỏ dấu tiếng Việt, ta có thể dùng phương pháp chuyển đổi unicode: tách tổ hợp ký tự (Normalization Form D) và loại bỏ các dấu kết hợp. Dưới đây, ta định nghĩa hàm remove_vietnamese_accents để thực hiện việc này.

In [ ]:
import unicodedata

def remove_vietnamese_accents(text: str) -> str:
    """
    Loại bỏ dấu tiếng Việt khỏi chuỗi văn bản.
    """
    # Normalize về dạng decomposed (NFD) để tách dấu khỏi ký tự
    text_nfd = unicodedata.normalize('NFD', text)
    # Loại bỏ các ký tự dấu (thuộc loại Mn - Mark, Nonspacing)
    text_no_diacritics = ''.join(ch for ch in text_nfd if unicodedata.category(ch) != 'Mn')
    # Thay thế chữ đ Đ đặc biệt
    text_no_diacritics = text_no_diacritics.replace('đ', 'd').replace('Đ', 'D')
    return text_no_diacritics

# Thử nghiệm hàm loại bỏ dấu trên một câu ví dụ
sample_text = "Phim kịch bản hấp dẫn, diễn xuất tuyệt vời!"
print("Trước khi loại bỏ dấu:", sample_text)
print("Sau khi loại bỏ dấu  :", remove_vietnamese_accents(sample_text))

Trước khi loại bỏ dấu: Phim kịch bản hấp dẫn, diễn xuất tuyệt vời!
Sau khi loại bỏ dấu  : Phim kich ban hap dan, dien xuat tuyet voi!


  Tiếp theo, áp dụng hàm trên cho cột nội dung đánh giá. Trong dataset, ta thấy có sẵn cột Noi_dung_sach_giu_dau (nội dung đã làm sạch và giữ dấu) và Noi_dung_sach (nội dung làm sạch không dấu). Ta sẽ sử dụng cột có dấu (giữ dấu) làm văn bản gốc. Sau đó tạo thêm cột text_noaccent chứa phiên bản không dấu tương ứng.

In [ ]:
# Sử dụng cột đã được làm sạch và giữ dấu làm văn bản chính
df['text'] = df['Noi_dung_sach_giu_dau'].astype(str)
# Tạo cột văn bản không dấu bằng cách áp dụng hàm đã định nghĩa
df['text_noaccent'] = df['text'].apply(remove_vietnamese_accents)

# Xem vài ví dụ về text gốc và text không dấu
for i in range(3):
    print(f"Ví dụ {i+1}:")
    print("Có dấu  :", df.loc[i, 'text'])
    print("Không dấu:", df.loc[i, 'text_noaccent'])


Ví dụ 1:
Có dấu  : coi rất cuon nha
Không dấu: coi rat cuon nha
Ví dụ 2:
Có dấu  : vô bổ phí tiền luẩn cuẩn không ra cái thể loại gì hoang đường vô lý không nên xem
Không dấu: vo bo phi tien luan cuan khong ra cai the loai gi hoang duong vo ly khong nen xem
Ví dụ 3:
Có dấu  : phim kịch bản lòng vòng đầu đuôi chuột bất hợp lý hài kiểu không có chiều sâu xem kiểu bị chán á thấy phí tiền ghê khuyên xem cho vui chứ đừng kì vọng nhiều
Không dấu: phim kich ban long vong dau duoi chuot bat hop ly hai kieu khong co chieu sau xem kieu bi chan a thay phi tien ghe khuyen xem cho vui chu dung ki vong nhieu


### 1.3 Gán nhãn số cho dữ liệu

  Mô hình phân loại cần nhãn dạng số. Ta chuyển các nhãn cảm xúc từ dạng text ("Tiêu cực", "Trung lập", "Tích cực") sang số tương ứng. Ví dụ, ta đặt:

Tiêu cực = 0

Trung lập = 1

Tích cực = 2

In [ ]:
# Ánh xạ nhãn text sang nhãn số
label_map = {"Tiêu cực": 0, "Trung lập": 1, "Tích cực": 2}
df['label'] = df['label_text'].map(label_map)

# Xác nhận thực hiện gán nhãn
print("Mapping nhãn:", label_map)
print("5 mẫu gán nhãn số:")
df[['text', 'label_text', 'label']].sample(5)


Mapping nhãn: {'Tiêu cực': 0, 'Trung lập': 1, 'Tích cực': 2}
5 mẫu gán nhãn số:


,text,label_text,label
693,phim thì ok nhưng mà hôm 21 02 mình có đi xem ...,Trung lập,1
4093,không có gì thất vọng dở như dự đoán,Tiêu cực,0
9181,phim quá dở luôn,Tiêu cực,0
639,nhận xét công tâm nha điểm mạnh quay đẹp diễn ...,Tích cực,2
6045,dở vãi chưởng,Tiêu cực,0


### 1.4 Chia dữ liệu thành tập Train/Validation/Test

  Bây giờ ta chia tập dữ liệu thành 3 phần: train (80%), validation (10%) và test (10%). Việc chia được thực hiện ngẫu nhiên nhưng có stratify theo nhãn để đảm bảo tỷ lệ các nhãn trong mỗi tập tương đương với toàn bộ dữ liệu (do bộ dữ liệu ban đầu đã cân bằng giữa các nhãn, stratify sẽ giúp giữ cân bằng). Quy trình:

- Chia dữ liệu ban đầu thành train (80%) và phần còn lại (20%).
- Từ phần 20% này tiếp tục chia đều thành validation (10%) và test (10%).

Quan trọng: Để tránh rò rỉ dữ liệu giữa các tập, chúng ta chỉ thực hiện việc sinh dữ liệu không dấu trên tập train. Tập validation và test giữ nguyên văn bản gốc có dấu để đánh giá mô hình trong điều kiện bình thường. (Trong quá trình huấn luyện, mô hình đã thấy cả dữ liệu không dấu, nên nó sẽ học được cách xử lý trường hợp thiếu dấu.)

In [ ]:
# Chia tập ban đầu thành train (80%) và temp (20%)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
# Chia tiếp temp thành val và test (mỗi phần ~10% tổng, tức là 50% của temp)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Kích thước tập train:", len(train_df))
print("Kích thước tập validation:", len(val_df))
print("Kích thước tập test:", len(test_df))
print("\nPhân bố nhãn trong mỗi tập:")
print("Train:\n", train_df['label_text'].value_counts())
print("Validation:\n", val_df['label_text'].value_counts())
print("Test:\n", test_df['label_text'].value_counts())


Kích thước tập train: 11522
Kích thước tập validation: 1440
Kích thước tập test: 1441

Phân bố nhãn trong mỗi tập:
Train:
 label_text
Trung lập    3841
Tiêu cực     3841
Tích cực     3840
Name: count, dtype: int64
Validation:
 label_text
Trung lập    480
Tích cực     480
Tiêu cực     480
Name: count, dtype: int64
Test:
 label_text
Tích cực     481
Trung lập    480
Tiêu cực     480
Name: count, dtype: int64


### 1.5 Tăng cường dữ liệu huấn luyện với bản không dấu

  Đối với tập huấn luyện (train_df), ta tiến hành nhân đôi dữ liệu bằng cách thêm phiên bản không dấu của mỗi mẫu:

- Lấy toàn bộ các mẫu trong train_df (văn bản có dấu và nhãn).
- Tạo một DataFrame bản sao nhưng văn bản được thay bằng text_noaccent.
- Kết hợp hai phần này lại với nhau để có tập huấn luyện mở rộng (train_aug_df).

Tập validation và test sẽ không được bổ sung bản không dấu (để giữ dữ liệu đánh giá độc lập).

In [ ]:
# Tạo tập dữ liệu train mở rộng bao gồm cả bản không dấu
train_df_copy = train_df.copy()
# DataFrame chứa văn bản có dấu (sử dụng cột 'text')
train_text_df = train_df_copy[['text', 'label']].copy()
# DataFrame chứa văn bản không dấu (đổi tên cột 'text_noaccent' thành 'text')
train_text_noaccent_df = train_df_copy[['text_noaccent', 'label']].copy()
train_text_noaccent_df.rename(columns={'text_noaccent': 'text'}, inplace=True)
# Kết hợp lại
train_aug_df = pd.concat([train_text_df, train_text_noaccent_df], ignore_index=True)
# Xáo trộn dữ liệu train_aug để trộn lẫn các mẫu có dấu và không dấu (tùy chọn)
train_aug_df = train_aug_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Số lượng mẫu tập train sau khi tăng cường:", len(train_aug_df))
# Xem vài dòng dữ liệu train sau tăng cường
train_aug_df.sample(5)


Số lượng mẫu tập train sau khi tăng cường: 23044


,text,label
5314,de ta thi ban bo tien ra mua ly nuoc suot 100k...,0
5766,diem cong duy nhat la dep con lai noi dung nhu...,0
6894,trải nghiệm rất tệ hại lạm dụng jumpscare vào ...,0
4742,trong phim mario maurer đảm nhận vai nat một a...,2
6421,cùng bthg nói chung là nhỏ khỉ đột cam có duyê...,1


Sau bước này, tập huấn luyện train_aug_df đã có cả nội dung có dấu và không dấu. Chúng ta đã sẵn sàng để đưa dữ liệu vào mô hình.

## Bước 2: Chuẩn bị tokenizer và mô hình PhoBERT

### 2.1 Khởi tạo tokenizer của PhoBERT

  Sử dụng AutoTokenizer của Huggingface để load tokenizer cho mô hình "vinai/phobert-base". Tokenizer này sẽ đảm nhiệm việc chuyển đổi câu tiếng Việt thành các token id phù hợp với PhoBERT. Lưu ý về tách từ: Ở đây ta chưa áp dụng tách từ thủ công cho tiếng Việt. Tokenizer PhoBERT gốc giả định đầu vào đã được tách từ bằng dấu gạch dưới giữa các âm tiết trong cùng một từ ​
[github.com](https://colab.research.google.com/drive/1QZoy6zUiPjezpULSiRqcYoam4E60JZjs#scrollTo=iBBNv-p_RQQz&line=2&uniqifier=1)
. Không tách từ có thể khiến việc token hóa kém tối ưu, nhưng mô hình vẫn có thể hoạt động (mặc dù có thể không đạt hiệu quả cao nhất). Để đơn giản, ta cứ sử dụng dữ liệu thô; trong thực tế có thể tích hợp bước tách từ như đề cập ở trên để cải thiện kết quả.

In [ ]:
# Import underthesea
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00


In [ ]:
from underthesea import word_tokenize

# Hàm tách từ tiếng Việt cho PhoBERT
def word_segment(text):
    """
    Tách từ tiếng Việt và nối bằng dấu gạch dưới (PhoBERT format).
    """
    return word_tokenize(text, format="text").replace(" ", "_")

# Hàm tokenize batch dữ liệu (có tách từ trước và kiểm tra kết quả tách)
def tokenize_batch(batch):
    segmented_texts = [word_segment(text) for text in batch['text']]
# In thử tối đa 5 dòng để kiểm tra
    for idx, (original_text, segmented_text) in enumerate(zip(batch['text'], segmented_texts)):
        if idx >= 5:  # chỉ in 5 mẫu đầu
            break
        print("Văn bản gốc     :", original_text)
        print("Văn bản đã tách:", segmented_text)
        print("="*50)
    # Token hóa văn bản đã tách từ
    return tokenizer(segmented_texts, truncation=True, max_length=256)


In [ ]:
# Khởi tạo tokenizer cho PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

### 2.2 Chuẩn bị dataset cho Huggingface Trainer

  Ta sẽ chuyển đổi các DataFrame thành định dạng datasets.Dataset của Huggingface để thuận tiện cho việc huấn luyện với Trainer. Sau đó, áp dụng tokenizer cho toàn bộ văn bản trong các tập train, validation, test.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
from datasets import Dataset

# Tạo Dataset từ DataFrame
train_dataset = Dataset.from_pandas(train_aug_df[['text', 'label']], preserve_index=False)
val_dataset   = Dataset.from_pandas(val_df[['text', 'label']], preserve_index=False)
test_dataset  = Dataset.from_pandas(test_df[['text', 'label']], preserve_index=False)

# Hàm tokenization cho batch dữ liệu
# def tokenize_batch(batch):
#     return tokenizer(batch['text'], truncation=True, max_length=256)

# Tokenize các dataset (áp dụng theo batch để nhanh hơn)
train_dataset = train_dataset.map(tokenize_batch, batched=True)
val_dataset   = val_dataset.map(tokenize_batch, batched=True)
test_dataset  = test_dataset.map(tokenize_batch, batched=True)

# Xóa cột text gốc sau khi đã token hóa (không cần thiết cho mô hình)
train_dataset = train_dataset.remove_columns("text")
val_dataset   = val_dataset.remove_columns("text")
test_dataset  = test_dataset.remove_columns("text")

# Kiểm tra một mẫu đã token hóa
print("Ví dụ mẫu sau token hóa:")
print(train_dataset[0])


Map:   0%|          | 0/23044 [00:00<?, ? examples/s]

Văn bản gốc     : quá nhiều yếu tố trung hoa trong một bộ phim mang tiếng văn hoá lịch sử việt nam người miền tây mặc áo bà ba chứ không phải áo cổ tàu kháng chiến chống pháp mà cộng sản không đc nhắc đến chỉ nhắc thiên địa hội với nghĩa hoà đoàn ai trong chúng ta cũng hiểu nếu làm phim về chiến tranh về lịch sử thì nên đưa những yếu tố thuần việt vào nhưng bộ phim này lại cài cắm rất nhiều yếu tố trung hoa phim có rất nhiều cảnh chém giết nhưng lại đc xếp loại không cũng hơi khó hiểu liệu có sự can thiệp nào đó để bộ phim đc tiếp cận với đối tượng khán giả nhỏ tuổi nhằm mục đích gì đó không thu gọn
Văn bản đã tách: quá_nhiều_yếu_tố_trung_hoa_trong_một_bộ_phim_mang_tiếng_văn_hóa_lịch_sử_việt_nam_người_miền_tây_mặc_áo_bà_ba_chứ_không_phải_áo_cổ_tàu_kháng_chiến_chống_pháp_mà_cộng_sản_không_đc_nhắc_đến_chỉ_nhắc_thiên_địa_hội_với_nghĩa_hòa_đoàn_ai_trong_chúng_ta_cũng_hiểu_nếu_làm_phim_về_chiến_tranh_về_lịch_sử_thì_nên_đưa_những_yếu_tố_thuần_việt_vào_nhưng_bộ_phim_này_lại_cài_cắm_rất_nhiều_

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Văn bản gốc     : cũng ổn không hấp dẫn lắm mùa này buôn bán quá đi coi cho đỡ buồn thôi
Văn bản đã tách: cũng_ổn_không_hấp_dẫn_lắm_mùa_này_buôn_bán_quá_đi_coi_cho_đỡ_buồn_thôi
Văn bản gốc     : phim như thể loại kinh dị thập niên 90 ngoài la hét ra thì các nhân vật như bị ngáo chả làm được gì
Văn bản đã tách: phim_như_thể_loại_kinh_dị_thập_niên_90_ngoài_la_hét_ra_thì_các_nhân_vật_như_bị_ngáo_chả_làm_được_gì
Văn bản gốc     : mình rất mê phim coi và cảm nhận tích cực trước giờ nhưng phim này bị rời rạc quá cảm xúc không tới các tình tiết rời rạc không ăn nhập với nhau từ lúc gặp đến yêu và kết hôn không có quá trình gì hết nhưng lại yêu thật lòng kết như vậy rất hợp lý chứ đến với nhau sống hạnh phúc thì nó kì hơn nữa 10 huhuthu gọn
Văn bản đã tách: mình_rất_mê_phim_coi_và_cảm_nhận_tích_cực_trước_giờ_nhưng_phim_này_bị_rời_rạc_quá_cảm_xúc_không_tới_các_tình_tiết_rời_rạc_không_ăn_nhập_với_nhau_từ_lúc_gặp_đến_yêu_và_kết_hôn_không_có_quá_trình_gì_hết_nhưng_lại_yêu_thật_lòng_kết_như_vậy_rất

Map:   0%|          | 0/1441 [00:00<?, ? examples/s]

Văn bản gốc     : bộ phim từ hơn 10 năm trước mà vẫn kín rạp không có từ nào khác để diễn tả ngoài siêu phẩm của nhân loại diễn xuất của diễn viên kỹ xảo và đặc biệt là nhạc phim thì không có từ nào để chê cả hihi
Văn bản đã tách: bộ_phim_từ_hơn_10_năm_trước_mà_vẫn_kín_rạp_không_có_từ_nào_khác_để_diễn_tả_ngoài_siêu_phẩm_của_nhân_loại_diễn_xuất_của_diễn_viên_kỹ_xảo_và_đặc_biệt_là_nhạc_phim_thì_không_có_từ_nào_để_chê_cả_hihi
Văn bản gốc     : nói sao ta hay do đô phim kinh dị của mình mạnh quá chọn vai diễn không phù hợp tt không hợp vai phản diện lúc diễn cứ bị hiền hiền á vai diễn đầu nên không đánh giá được tt nhưng mà để chọn diễn viên diễn tròn vai thì nên chọn diễn viên khác vì tt chưa có diễn được nét mặt kiểu sẽ hận thù á với lại phim lợi dụng độc thoại nhiều quá nó không có nhiều tình tiết sẽ khiến người xem sợ như phim kinh dị cho lắm mình thấy hù chưa đã mình coi mà bạn mình ngồi cười luôn á như đợt mình coi phim tee yod của thái ấy coi xong bộ phim mà mình khờ luôn vì người t

  Sau khi token hóa, mỗi mẫu dữ liệu bây giờ gồm các trường: input_ids, attention_mask và label. Trường input_ids là chuỗi các token id của PhoBERT, attention_mask đánh dấu vị trí token thật (1) hay padding (0), và label là nhãn lớp.

### 2.3 Khởi tạo mô hình phân loại PhoBERT

  Sử dụng AutoModelForSequenceClassification để tạo một mô hình phân loại dựa trên PhoBERT. Mô hình này sẽ tự thêm một tầng phân loại (dense) phía trên PhoBERT để dự đoán 3 nhãn.
  Ta truyền đối số num_labels=3 để cấu hình số nhãn đầu ra cho phù hợp với bài toán (Tiêu cực/Trung lập/Tích cực).

In [ ]:
# Khởi tạo mô hình PhoBERT cho classification (3 nhãn)
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  PhoBERT-base có ~135 triệu tham số. Khi khởi tạo, các tham số của phần transformer sẽ được load từ pre-trained PhoBERT, còn các tham số của layer phân loại mới thêm được khởi tạo ngẫu nhiên.
   Mô hình PhoBERT bao gồm sẵn các cơ chế Dropout (thường tỷ lệ ~0.1) ở các tầng và trong lớp phân loại, giúp giảm overfitting. Ta sẽ giữ nguyên các cấu hình mặc định này.

## Bước 3: Huấn luyện mô hình

Sử dụng API Trainer của Huggingface để huấn luyện giúp đơn giản hóa nhiều công đoạn như vòng lặp epoch, tính toán loss/accuracy, v.v.

### 3.1 Thiết lập thông số huấn luyện (TrainingArguments)

  Các tham số quan trọng bao gồm:
output_dir: thư mục lưu mô hình và checkpoint.

- epochs: số epoch tối đa (ví dụ 10, sẽ dừng sớm nếu dùng EarlyStopping).

- batch_size: kích thước batch cho train và eval (phụ thuộc tài nguyên GPU, đặt 16 cho train, 32 cho eval).

- learning_rate: tốc độ học, đặt giá trị nhỏ (2e-5) phù hợp cho fine-tune BERT.

- weight_decay: hệ số regularization L2 (ví dụ 0.01) để giúp tránh overfitting.
evaluation_strategy: chọn "epoch" để thực hiện đánh giá trên tập validation mỗi epoch.

- load_best_model_at_end: tự động load mô hình tốt nhất (theo metric) sau khi train xong.

- metric_for_best_model: chọn metric để xác định mô hình tốt nhất (ở đây ta chọn "eval_accuracy").

- save_strategy: chọn lưu checkpoint mỗi epoch (kết hợp với load_best_model_at_end để có mô hình tốt nhất).

- early_stopping: sẽ thiết lập qua callback bên dưới.

In [ ]:
!pip install --upgrade transformers

In [ ]:
# Thiết lập các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./phobert_sentiment",    # thư mục lưu kết quả
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,                   # sử dụng weight decay để regularization
    eval_strategy="epoch",         # đánh giá trên val mỗi epoch
    save_strategy="epoch",               # lưu mô hình mỗi epoch
    load_best_model_at_end=True,         # tự động load mô hình tốt nhất sau train
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,              # chọn mô hình có accuracy cao nhất
    logging_dir="./train_logs",          # thư mục logging (tùy chọn)
    logging_steps=50,                    # tần suất log (nếu muốn log chi tiết hơn)
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### 3.2 Định nghĩa hàm tính toán metric cho validation

Để Trainer tự động tính và báo cáo độ chính xác (accuracy) trên tập validation, ta cần cung cấp hàm compute_metrics. Hàm này nhận đầu ra mô hình và nhãn thật, sau đó tính toán các metrics cần thiết. Ở đây ta chỉ tính accuracy.

In [ ]:
# Định nghĩa hàm tính toán accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}


### 3.3 Khởi tạo Trainer với mô hình, dữ liệu và callback EarlyStopping

  Thiết lập một EarlyStoppingCallback để dừng huấn luyện sớm nếu model không cải thiện trên tập validation:

- early_stopping_patience=3 nghĩa là nếu 3 lần đánh giá liên tiếp không thấy accuracy tăng thì dừng.

- early_stopping_threshold=0.0 nghĩa là yêu cầu tăng ít nhất một lượng dương (bất kỳ) thì mới tính là cải thiện.

Chú ý truyền tokenizer=tokenizer để Trainer tự động sử dụng trong việc tạo batch (đệm padding động).

In [ ]:
# Khởi tạo EarlyStopping callback
early_stop = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0)

# Tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stop]
)


<ipython-input-17-bb6227c79d37>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### 3.4 Tiến hành huấn luyện

  Bắt đầu train bằng cách gọi trainer.train(). Quá trình huấn luyện sẽ in ra màn hình các thông tin mỗi epoch, bao gồm loss trên tập train, loss và accuracy trên tập validation. Nếu có GPU, Trainer sẽ tự động sử dụng GPU để tăng tốc (kiểm tra bằng torch.cuda.is_available()). Thời gian huấn luyện tùy thuộc vào kích thước dữ liệu và cấu hình. Với ~23k mẫu train (sau augmentation) và batch size 16, thời gian train trên GPU có thể vài phút mỗi epoch.

In [ ]:
# Huấn luyện mô hình
train_results = trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.444600,0.273070,0.906944
2,0.260100,0.136931,0.968056
3,0.149200,0.080863,0.984722
4,0.104300,0.111142,0.982639
5,0.039800,0.196441,0.974306
6,0.025900,0.127402,0.983333


  Trong quá trình chạy, ta theo dõi:

- loss: hàm mất mát trên tập huấn luyện (giảm dần qua các batch).

- eval_loss và eval_accuracy: hàm mất mát và độ chính xác trên tập validation mỗi epoch (dùng để xác định early stopping và mô hình tốt nhất).

Nhờ EarlyStopping, training sẽ dừng sớm nếu model bắt đầu overfit. Nhờ load_best_model_at_end=True, sau khi dừng, trainer.model sẽ là trạng thái của mô hình tốt nhất trong các epoch.
Sau khi huấn luyện xong, ta lưu lại mô hình tốt nhất ra thư mục (kèm tokenizer) để có thể sử dụng sau này.

In [ ]:
# Lưu lại mô hình đã được fine-tune (tốt nhất) và tokenizer
trainer.save_model("./best_phobert_model")
tokenizer.save_pretrained("./best_phobert_model")

# In ra kết quả mô hình tốt nhất trên tập validation
best_acc = trainer.state.best_metric
print(f"Độ chính xác tốt nhất trên tập validation: {best_acc:.4f}")
print("Checkpoint mô hình tốt nhất được lưu tại:", trainer.state.best_model_checkpoint)


Độ chính xác tốt nhất trên tập validation: 0.9847
Checkpoint mô hình tốt nhất được lưu tại: ./phobert_sentiment/checkpoint-4323


## Bước 4: Đánh giá mô hình trên tập kiểm tra (Test)

  Sau khi huấn luyện, ta sử dụng mô hình tốt nhất để dự đoán trên tập test (10% dữ liệu chưa hề dùng trong train/val). Mục đích là đánh giá chất lượng tổng thể của mô hình trên dữ liệu hoàn toàn mới. Ta sẽ tính độ chính xác (accuracy) trên tập test, đồng thời in classification report để xem chi tiết hơn (precision, recall cho từng lớp).

In [ ]:
# Đánh giá mô hình trên tập test
test_results = trainer.predict(test_dataset)
pred_labels = np.argmax(test_results.predictions, axis=1)
true_labels = test_results.label_ids

# Tính độ chính xác
test_acc = accuracy_score(true_labels, pred_labels)
print(f"Độ chính xác (accuracy) trên tập test: {test_acc:.4f}")

# Báo cáo chi tiết theo từng lớp
class_names = ["Tiêu cực", "Trung lập", "Tích cực"]
print("Báo cáo phân loại trên tập test:")
print(classification_report(true_labels, pred_labels, target_names=class_names))


Độ chính xác (accuracy) trên tập test: 0.9820
Báo cáo phân loại trên tập test:
              precision    recall  f1-score   support

    Tiêu cực       0.98      1.00      0.99       480
   Trung lập       0.98      0.99      0.99       480
    Tích cực       0.99      0.96      0.97       481

    accuracy                           0.98      1441
   macro avg       0.98      0.98      0.98      1441
weighted avg       0.98      0.98      0.98      1441



  Giải thích kết quả:

- Accuracy trên tập test cho biết tỷ lệ dự đoán đúng tổng thể của mô hình.

- Classification report cho biết độ chính xác (precision), độ phủ (recall) và điểm F1 cho từng nhãn. Ta kỳ vọng mô hình đạt kết quả cao và cân bằng trên ba lớp do dữ liệu huấn luyện đã cân bằng và mô hình đã được fine-tune tốt.

Cuối cùng, ta đã xây dựng thành công một mô hình PhoBERT fine-tune để phân loại cảm xúc cho các bài đánh giá phim. Mô hình có thể được dùng để dự đoán cảm xúc của các đánh giá mới. Ví dụ, ta có thể thử dự đoán nhanh trên một câu bất kỳ:

In [ ]:
import os
import shutil

# Tạo thư mục mới nếu chưa có
save_dir = "best_phobert_model"
os.makedirs(save_dir, exist_ok=True)

# Danh sách các file cần chuyển vào thư mục
model_files = [
    "config.json",
    "model.safetensors",
    "tokenizer_config.json",
    "vocab.txt",
    "bpe.codes",
    "special_tokens_map.json",
    "training_args.bin",
    "added_tokens.json"
]

# Di chuyển các file vào thư mục
for file in model_files:
    if os.path.exists(file):
        shutil.move(file, os.path.join(save_dir, file))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer + model từ thư mục mới tạo
model_dir = "./best_phobert_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# Đưa lên GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Danh sách nhãn
class_names = ["Tiêu cực", "Trung lập", "Tích cực"]


In [ ]:
# # Hàm tách từ (dùng lại như lúc huấn luyện)
# def word_segment(text):
#     from underthesea import word_tokenize
#     return word_tokenize(text, format="text").replace(" ", "_")

# # Câu đánh giá mới
# new_review = "bộ phim này bình thường không có gì để xem"

# # Tách từ trước khi tokenize
# segmented_review = word_segment(new_review)

# # Tokenize + move to GPU
# inputs = tokenizer(segmented_review, return_tensors="pt")
# inputs = inputs.to(model.device)

# # Dự đoán
# outputs = model(**inputs)
# predicted_label = outputs.logits.argmax(dim=1).item()

# # In kết quả
# print("Đánh giá (gốc):", new_review)
# print("Đánh giá (đã tách):", segmented_review)
# print("Dự đoán cảm xúc:", class_names[predicted_label])


Đánh giá (gốc): bộ phim này bình thường không có gì để xem
Đánh giá (đã tách): bộ_phim_này_bình_thường_không_có_gì_để_xem
Dự đoán cảm xúc: Tiêu cực


In [ ]:
import torch.nn.functional as F

# Hàm tách từ (dùng lại như lúc huấn luyện)
def word_segment(text):
    from underthesea import word_tokenize
    return word_tokenize(text, format="text").replace(" ", "_")

# Câu đánh giá mới
new_review = "Mot bo phim binh thuong, khong hay cung khong do"

# Tách từ
segmented_review = word_segment(new_review)

# Tokenize + move to GPU
inputs = tokenizer(segmented_review, return_tensors="pt").to(model.device)

# Dự đoán
outputs = model(**inputs)

# Tính xác suất (softmax)
probs = F.softmax(outputs.logits, dim=1).squeeze().tolist()

# Nhãn dự đoán
predicted_label = torch.argmax(outputs.logits, dim=1).item()

# In kết quả
print("Đánh giá (gốc):", new_review)
print("Đánh giá (đã tách):", segmented_review)
print("→ Dự đoán cảm xúc:", class_names[predicted_label])
print("→ Xác suất từng lớp:")
for i, name in enumerate(class_names):
    print(f"  - {name:<10}: {probs[i]*100:.2f}%")


Đánh giá (gốc): Mot bo phim binh thuong, khong hay cung khong do
Đánh giá (đã tách): Mot_bo_phim_binh_thuong_,_khong_hay_cung_khong_do
→ Dự đoán cảm xúc: Trung lập
→ Xác suất từng lớp:
  - Tiêu cực  : 1.59%
  - Trung lập : 98.08%
  - Tích cực  : 0.33%
